In [ ]:
!pip install -q openai==0.28 tqdm gdown

In [ ]:
import csv
import requests
import json
import time
import openai
import os
from tqdm.notebook import tqdm
from google.colab import drive
import os
import zipfile
import gdown
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.colab import auth
from IPython.display import clear_output

## query_openai_api.py

In [ ]:
from chat_prompts import chat_prompts

def ask_gpt_chat(model_name, sys_prompt, user_prompt, temperature=0, top_p=1):
    MAX_API_RETRY = 5
    for i in range(MAX_API_RETRY):
        try:
            response = openai.ChatCompletion.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": sys_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                top_p=top_p
            )
            return response["choices"][0]["message"]["content"]
        except Exception as e:
            print(f"Error: {e}. Retry {i + 1}/{MAX_API_RETRY}")
            time.sleep(5 * (i + 1))
    return None

In [ ]:
def get_question_text(prompt_type, question):
    return 'I ' + question['text_first_person'].lower()

In [ ]:
def read_ocean(filename, ipip):
    ocean_data = []
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for row in csvreader:
            if ipip == '120':
                text_first_person, text_second_person, label, key = row[4], row[5], row[7], row[9]
            elif ipip == '300':
                text_first_person, text_second_person, label, key = row[4], row[5], row[6], row[8]
            ocean_data.append({
                'text_first_person': text_first_person,
                'text_second_person': text_second_person,
                'label': label,
                'key': key
            })
    return ocean_data

In [ ]:
if __name__ == '__main__':
    ipip_dataset = '300'
    filename = '/Tests/ocean_' + ipip_dataset + '_corrected.csv'
    ocean_data = read_ocean(filename, ipip_dataset)

    model_name = 'gpt-3.5-turbo'
    temperature = 0.01
    top_p = 1
    max_tokens = 120

    output_directory = '/Data/Outputs_' + ipip_dataset + '/' + model_name.replace('/', '-') + '/'
    os.makedirs(output_directory, exist_ok=True)

    #START EXPERIMENTS
    for prompt_type in chat_prompts.keys():
        output_filename = output_directory + prompt_type + '.json'

        system_message = chat_prompts[prompt_type]['system_message']
        user_message = chat_prompts[prompt_type]['user_message']

        output_dict = {
            'system_prompt': system_message,
            'user_prompt': user_message,
            'responses': []
        }

        for q, question in enumerate(ocean_data):
            print(prompt_type, q)

            question_text = get_question_text(prompt_type, question)
            system_prompt = system_message.replace('{item}', question_text)
            user_prompt = user_message.replace('{item}', question_text)

            response = None
            while not response:
                response = ask_gpt_chat(model_name, system_prompt, user_prompt)

            #store question and response
            question['input_prompt_system'] = system_prompt
            question['input_prompt_user'] = user_prompt
            question['processed_response'] = response

            output_dict['responses'].append(question)

            json_data = json.dumps(output_dict)
            with open(output_filename, "w") as file:
                file.write(json_data)

            with open(output_filename, "w", encoding="utf-8") as f:
                json.dump(output_dict, f, indent=2, ensure_ascii=False)

#query_llama_api

In [ ]:
def ask_openrouter(model_name, input_prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = None
            response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
                  data=json.dumps({
            "model": "meta-llama/llama-2-70b-chat",
            "messages": [{"role": "user", "content": input_prompt}],

                })
              )
            if response.status_code == 200:
                content = response.json()["choices"][0]["message"]["content"]
                return content
            else:
                print(f"Attempt {attempt + 1} failed with status code {response.status_code}")
                time.sleep(2)

        except Exception as e:
            print(f"Attempt {attempt + 1} failed with error: {str(e)}")
            time.sleep(2)

    return ""

In [ ]:
if __name__ == '__main__':
    ipip_dataset = '300'
    filename = '/Tests/ocean_' + ipip_dataset + '_corrected.csv'
    ocean_data = read_ocean(filename, ipip_dataset)

    model_name = 'meta-llama/llama-2-70b-chat'
    temperature = 0.01
    top_p = 1
    max_tokens = 120

    output_directory = '/Data/Outputs_' + ipip_dataset + '/' + model_name.replace('/', '-') + '/'
    os.makedirs(output_directory, exist_ok=True)

    #START EXPERIMENTS
    for prompt_type in chat_prompts.keys():
        output_filename = output_directory + prompt_type + '.json'

        system_message = chat_prompts[prompt_type]['system_message']
        user_message = chat_prompts[prompt_type]['user_message']

        output_dict = {}
        output_dict['system_prompt'] = system_message
        output_dict['user_prompt'] = user_message
        output_dict['responses'] = []

                ## Llama prompt type 2 - original
        input_message = "<s>[INST] <<SYS>>\n"
        input_message += system_message.strip() + "\n"
        input_message += "<</SYS>>\n\n"
        input_message += user_message.strip() + " [/INST]"

        for q, question in enumerate(ocean_data):
            start_time = time.time()
            question_text = get_question_text(prompt_type, question)
            input_prompt = input_message.replace('{item}', question_text)

            input_dict = {
                "prompt": input_prompt,
                "temperature": temperature,
                "top_p": top_p,
                "max_tokens": max_tokens
            }

            #store question and response
            response = ask_openrouter(model_name, input_prompt)
            question['input_prompt'] = input_prompt
            question['response'] = response
            question['processed_response'] = response.replace(input_prompt, '')
            print(response)
            output_dict['responses'].append(question)

            json_data = json.dumps(output_dict)
            with open(output_filename, "w") as file:
                file.write(json_data)